## Flower Classification
In this programming assignment, we will re-visit the Flower Classification task and solve it using deep neural networks, more specifically
Convolutional Neural Networks. Recap that the flower images are divided into five classes: chamomile, tulip, rose, sunflower, dandelion.
For each class there are about 800 photos. Photos are not high resolution, about 320x240 pixels. Photos are not reduced to a single size, they have different proportions!

You are free to use any deep learning frameworks to build, train and evaluate your models, e.g. PyTorch, Tensorflow 2.x(or its high level API Keras), MXNet and etc. You are free to use already-built models from these libraries like ResNet-50, InceptionNet-V2, and etc, and adding customized classification layer on top of it for your own purpoose.

For computational resource, you will be required to use GPU to train your neural network. Luckily, Google Colab provides free GPU option, you can use that, though it's not very powerful(limited computational speed and limited memory). But in general, for relatively small deep networks(e.g, VGG-16) and small batch size, you can get decent baseline results faster. So always try smaller networks first, and try to select appropriate batch size such that your GPU memories don't overload. 

You are free to use pretrained weights from these established models, and apply transfer learning method by either fixing the weight of pretrained weights and only training on the classification head, or use fine-tuning strategy(small learning rate on whole network.) Transfer learning strategy can accelerate your training progress a lot, you might want to use the strategy when you use those larger networks(i.e. ResNet-50, InceptionNet etc).

To help you detect whether your network and training pipelines are built correctly, a common strategy is to train your model on a very small portion of your training set for a few epoches(should be pretty fast), and to check whether the model can be overfitted(achiving very high accuracy) on this small portion. If the model overfits, then it means that your pipeline setup has no problem, then you can proceed to start your whole training task.

Submit your results to Kaggle competition: https://www.kaggle.com/c/ee596-flower-classification-deep-neural-networks/

## Step 0: Import Necessary Modules
Here we use PyTorch as an example, you can use whatever libraries we want. Torch is the basic backend for the pytorch framework, and torchvision provides you with image proocessing methods before applied to deep neural networks.

In [1]:
# mount the drive so that I can import the datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torchvision
import torchvision.transforms as transform
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, ConcatDataset, random_split
from sklearn.model_selection import train_test_split


import torch.nn as nn
import torchvision.models as models
from torchvision.utils import make_grid
import torch.nn.functional as F

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

import time
import copy

classes = ('daisy', 'dandelion', 'rose', 'sunflower', 'tulip')

## Step1: Data Preprocessing and Loading
Here in order to provide the image tensor that can be input to your network, the following basic steps you need to
follow. 
1. Resize all images to the same resolution which is required by your neural network. 
2. Normalize your pixel values to small range, say -1 to 1. 

This 2 steps are the most basic steps in feeding images to a CNN, but you can add more on top of it, which is called image augmentation. Image augmentation is a process where you randomly choose a particular transform or perturbation on your original image(Flip, rotate, crop, add noise etc) such that you are creating more samples for your training, which potentially increase the robustness of your model.

The easiest way for you to start with preprocessing is using the built-in transforms in the torchvision module.

In [3]:
simple_transform = transform.Compose([
                                 transform.Resize((220, 220)),
                                 transform.ToTensor(), 
                                 #transform.Normalize((0.4124234616756439, 0.3674212694168091, 0.2578217089176178), 
                                  #                   (0.3268945515155792, 0.29282665252685547, 0.29053378105163574))])
                                transform.Normalize(mean=[0.485, 0.456, 0.406],
                                                    std=[0.229, 0.224, 0.225])])

# define inception transform to be the same as simple with different image resize for 299x299 to match inception net architecture
inception_transform = transform.Compose([
                                 transform.Resize((299, 299)),
                                 transform.ToTensor(), 
                                 transform.Normalize((0.4124234616756439, 0.3674212694168091, 0.2578217089176178), 
                                                     (0.3268945515155792, 0.29282665252685547, 0.29053378105163574))])

Before Loading, you will create a Dataset class to iterate through your image data. Here we use ImageFolder dataset
which is sub-class of Dataset class for you to iterate through images. Check out torch.utils.Dataset to see how you
can define your own dataset to iterate through images.

In [ ]:
# Flower Test Location
test_path ='/content/drive/My Drive/UW/ML_596/PA5/flower_test'

In [4]:
# Flower Train Location
path = '/content/drive/My Drive/UW/ML_596/PA5/flower_train'
original = ImageFolder(path, transform=simple_transform)
train, val = train_test_split(original, test_size=0.2, random_state=43)

Create the dataloader such that your data can be fed into the network in batches

In [5]:
# Number of workers enable parallel loading
bs = 16 # Batch size
loaders = {
    'train': DataLoader(train, batch_size=bs, num_workers=2, pin_memory=True),
    'val': DataLoader(val, batch_size=bs, num_workers=2, pin_memory=True),
}

dataset_sizes = {
    'train': len(train),
    'val': len(val)
}

## Step 2: Define the evaluation metrics(Accuracy)

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1) 
    return torch.tensor(torch.sum(preds == labels).item() / len(preds)), preds

##  Step 3: Training Pipeline - Train and validation each epoches, plot curves etc.

In [ ]:
def train(epochs, model):
    """
    
    """
    ## Step 1: Send model to GPU device if GPU available, creating Loss function.
    print('Creating a model...')
    device= torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    ## Step 2: Create Optimizer, here we use pre-built and pretrained models like VGG-19, we use transfer
    ## learning so only train the classifier part or fully connected part while keeping the backbone of 
    ## the network weight fixed. Adam is the gradient descent algoritm we want to use as default.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

    since = time.time()
    best_model = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    losses = {'train' : [], 'val' : [] }
    accuracies = {'train' : [], 'val' : [] }
    lr = []    

    ## Loop for train and val for every epoch
    for epoch in range(epochs):
        for phase in ['train', 'val']:
            
            ## Mode activation, before training network, you must call model.train(), before validation, you must call model.eval()
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0.0
            
            ## Loading data samples from your data loaders and feeding them into the network for training
            for inputs, labels in loaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                ## Clear the gradient for last batch
                optimizer.zero_grad()

                ## Only record gradient when training, validation only forward pass the tensor, no gradient information required, 
                ## so fewer memory need to be allocated.
                with torch.set_grad_enabled(phase=='train'):
                    outp = model(inputs)
                    #print(outp)
                    if model == inception_v3_model and phase=='train':
                      _, pred = torch.max(outp.logits, 1)
                      loss = criterion(outp.logits, labels)
                    else:
                      _, pred = torch.max(outp, 1)
                      loss = criterion(outp, labels)

                    if phase == 'train':
                        ## Back propagation and gradient update
                        loss.backward()
                        optimizer.step()
                ## Sum up the loss and compute accuracy.
                running_loss += loss.item()*inputs.size(0)
                running_corrects += torch.sum(pred == labels.data)

            if phase == 'train':
                acc = 100. * running_corrects.double() / dataset_sizes[phase]
                scheduler.step(acc)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double()/dataset_sizes[phase]
            losses[phase].append(epoch_loss)
            accuracies[phase].append(epoch_acc)
      
            if phase == 'train':
                print('Epoch: {}/{}'.format(epoch+1, epochs))
                
                
            print('{} - loss: {}, accuracy: {}'.format(phase, epoch_loss, epoch_acc))
            lr.append(scheduler._last_lr)
        
            if phase == 'val':
                print('Time: {}m {}s'.format((time.time()- since)//60, (time.time()- since)%60))
                print('=='*31)
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model.state_dict())
                
    time_elapsed = time.time() - since
    print('CLASSIFIER TRAINING TIME {}m {}s'.format(time_elapsed//60, time_elapsed%60))
    print('=='*31)
    model.load_state_dict(best_model)

## Step 4: Initiate Your model

In [6]:
vgg19_bn = torchvision.models.vgg19_bn(pretrained=True)
for param in vgg19_bn.parameters():
    param.grad_requires = False

vgg19_bn.classifier[6] = nn.Linear(4096, len(original.classes), bias=True)


'''
inception_v3 model instantiation so that train code will run
'''
inception_v3_model = torchvision.models.inception_v3(pretrained=True)

# change last layer to output only 5 classes
inception_v3_model.fc = nn.Linear(in_features=2048, out_features=len(classes), bias=True)

Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

## Step 5: Start Training

In [ ]:
epochs = 20
train(epochs=epochs, model=vgg19_bn)

#save the model
torch.save(vgg19_bn.state_dict(), '/content/drive/My Drive/UW/ML_596/PA5/models/vgg19_bn.pth')

Creating a model...


KeyboardInterrupt: ignored

# Train Resnet50, 101, and Inception v3 models

---




In [ ]:
'''
Transfer Learning with Resnet50 architecture. 
'''
resnet50_model = torchvision.models.resnet50(pretrained=True)

# change last layer to output only 5 classes
resnet50_model.fc = nn.Linear(in_features=2048, out_features=len(classes), bias=True)

#print architecture
'''
for name, child in resnet50_model.named_children():
  print(name,child)
'''

for name, child in resnet50_model.named_children():
    if name in ['fc','layer4']:
        print(name + ' is unfrozen')
        for param in child.parameters():
            param.requires_grad = True
    else: 
        for param in child.parameters():
            param.requires_grad = False
print('-- all other layers remain frozen\n')


#train for 20 epochs
epochs = 20
train(epochs=epochs, model=resnet50_model)

#save the model weights with a specific name
torch.save(resnet50_model.state_dict(), '/content/drive/My Drive/UW/ML_596/PA5/models/resnet50_fc_4_normalize.pth')

layer4 is unfrozen
fc is unfrozen
-- all other layers remain frozen

Creating a model...
Epoch: 1/20
train - loss: 0.5907909627713747, accuracy: 0.8029699384281057
val - loss: 0.3812705560462002, accuracy: 0.8885672937771346
Time: 15.0m 40.01428818702698s
Epoch: 2/20
train - loss: 0.2849847336192789, accuracy: 0.9043824701195219
val - loss: 0.2734691033573813, accuracy: 0.9073806078147613
Time: 31.0m 44.82335686683655s
Epoch: 3/20
train - loss: 0.12792832724182007, accuracy: 0.9579862368706991
val - loss: 0.2891861430750742, accuracy: 0.9044862518089725
Time: 47.0m 49.42583656311035s


In [ ]:
'''
Transfer Learning with Resnet101 architecture. 
'''

resnet101_model = torchvision.models.resnet101(pretrained=True)

# change last layer to output only 5 classes
resnet101_model.fc = nn.Linear(in_features=2048, out_features=len(classes), bias=True)

#print architecture
'''
for name, child in resnet101_model.named_children():
  print(name,child)
'''

for name, child in resnet101_model.named_children():
    if name in ['fc','layer4']:
        print(name + ' is unfrozen')
        for param in child.parameters():
            param.requires_grad = True
    else: 
        for param in child.parameters():
            param.requires_grad = False
print('-- all other layers remain frozen\n')

#train for 20 epochs
epochs = 25
train(epochs=epochs, model=resnet101_model)

#save the model weights with a specific name
torch.save(resnet101_model.state_dict(), '/content/drive/My Drive/UW/ML_596/PA5/models/resnet101_model_fc_4_normalize.pth')

layer3 is unfrozen
layer4 is unfrozen
fc is unfrozen
-- all other layers remain frozen

Creating a model...
Epoch: 1/25
train - loss: 0.9330185645302755, accuracy: 0.6638898949655921
val - loss: 0.5799604189378653, accuracy: 0.8031837916063675
Time: 1.0m 44.55103778839111s
Epoch: 2/25
train - loss: 0.559387862153435, accuracy: 0.8084027526258603
val - loss: 0.5134212152140185, accuracy: 0.829232995658466
Time: 3.0m 28.707796096801758s
Epoch: 3/25
train - loss: 0.39631072818498947, accuracy: 0.8587468308583847
val - loss: 0.5484407767545643, accuracy: 0.8118668596237336
Time: 5.0m 12.69052791595459s
Epoch: 4/25
train - loss: 0.2756995708290022, accuracy: 0.9076421586381745
val - loss: 0.6225760234075068, accuracy: 0.8104196816208393
Time: 6.0m 56.56160068511963s
Epoch: 5/25
train - loss: 0.22239992575708484, accuracy: 0.9253893516841724
val - loss: 0.7735151414760805, accuracy: 0.7959479015918958
Time: 8.0m 40.446136713027954s
Epoch: 6/25
train - loss: 0.18156124354326306, accuracy: 0.9

In [ ]:
'''
Since Inception v3 uses 299x299 image shape, need to reload the data and use inception transform
'''

# load the images with ImageFolder with the inception transform
path = '/content/drive/My Drive/UW/ML_596/PA5/flower_train'
original = ImageFolder(path, transform=inception_transform)
train_, val_ = train_test_split(original, test_size=0.2, random_state=43)

# Generate the DataLoaders
bs = 16 # Batch size
loaders = {
    'train': DataLoader(train_, batch_size=bs, num_workers=2, pin_memory=True),
    'val': DataLoader(val_, batch_size=bs, num_workers=2, pin_memory=True),
}

dataset_sizes = {
    'train': len(train_),
    'val': len(val_)
}

In [ ]:
# inception_v3 model train

inception_v3_model = torchvision.models.inception_v3(pretrained=True)

# change last layer to output only 5 classes
inception_v3_model.fc = nn.Linear(in_features=2048, out_features=len(classes), bias=True)

#print architecture
#for name, child in inception_v3_model.named_children():
#  print(name,child)

for name, child in inception_v3_model.named_children():
    if name in ['fc', 'Mixed_7c', 'Mixed_7b']:
        print(name + ' is unfrozen')
        for param in child.parameters():
            param.requires_grad = True
    else: 
        for param in child.parameters():
            param.requires_grad = False
print('-- all other layers remain frozen\n')


#train for 25 epochs
epochs = 25
train(epochs=epochs, model=inception_v3_model)

#save the model weights
torch.save(inception_v3_model.state_dict(), '/content/drive/My Drive/UW/ML_596/PA5/models/inception_v3_3.pth')

## Step 6: Evaluation on your test set, show the classification accuracy, confusion matrix, do some simple analysis on which flower categories are easily misclassified.

evaluate method performs forward pass on validation set and computes accuracy by class, total accuracy and converts the prediction and label tensors to numpy lists that are returned so that sklearn can be easily implemented for accuracy score, confusion matrix and classification matrix

In [7]:
def evaluate(model_):
  # call model.eval() to prepare for evalutation
  model_.eval()

  correct_pred = {classname : 0 for classname in classes}
  total_pred = {classname : 0 for classname in classes}
  total_accuracy = 0.0
  y_pred_tensors = []
  y_true_tensors = []

  with torch.no_grad():
    for data in loaders['val']:
        
        images, labels = data
        outputs = model_(images.float()) #run batch through the netork
        _, preds = torch.max(outputs, 1)
        
        y_pred_tensors.append(preds)
        y_true_tensors.append(labels)

        for label, prediction in zip(labels, preds):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

  print('Accuracy by Class')
  for classname, num_correct in correct_pred.items():
    acc = float(num_correct) / total_pred[classname] * 100
    total_accuracy += acc
    print('{:15s}: {:.2f}%'.format(classname, acc))
  print(f'Total Accuracy = {total_accuracy/len(classes)}%')

  # lists to hold numpy values
  y_pred_numpy = []
  y_true_numpy = []

  # convert y_pred tensor to numpy list elementwise
  for p in y_pred_tensors:
    p = p.numpy()
    for x in p:
      y_pred_numpy.append(x)

  # convert y_true tensor to numpy list elementwise
  for t in y_true_tensors:
    t = t.numpy()
    for x in t:
      y_true_numpy.append(x)

  return y_pred_numpy, y_true_numpy


def eval_analysis(model):
  y_pred, y_true = evaluate(model)

  # confusion matrix on y_true vs y_pred
  print('\nConfusion Matrix: \n', confusion_matrix(y_true, y_pred))

  # accuracy of y_true vs y_pred
  print('\nTotal Accuracy: \n', accuracy_score(y_true, y_pred))

  # classification report
  print('\nClassification Report: \n', classification_report(y_true, y_pred))

In [ ]:
print('Load model with weights')
vgg19_bn.load_state_dict(torch.load('/content/drive/My Drive/UW/ML_596/PA5/models/vgg19_bn.pth'))

eval_analysis(vgg19_bn)

Accuracy by Class
daisy          : 83.81%
dandelion      : 83.87%
rose           : 81.95%
sunflower      : 81.90%
tulip          : 90.12%
Total Accuracy = 84.33271949287794%

Confusion Matrix: 
 [[ 88   3   5   3   6]
 [ 14 156   2   9   5]
 [  3   0 109   0  21]
 [  4   4   5  86   6]
 [  2   2   9   3 146]]

Total Accuracy: 
 0.8465991316931982

Classification Report: 
               precision    recall  f1-score   support

           0       0.79      0.84      0.81       105
           1       0.95      0.84      0.89       186
           2       0.84      0.82      0.83       133
           3       0.85      0.82      0.83       105
           4       0.79      0.90      0.84       162

    accuracy                           0.85       691
   macro avg       0.84      0.84      0.84       691
weighted avg       0.85      0.85      0.85       691



In [ ]:
resnet50_model = torchvision.models.resnet50(pretrained=True)
# change last layer to output only 5 classes
resnet50_model.fc = nn.Linear(in_features=2048, out_features=len(classes), bias=True)

resnet50_model.load_state_dict(torch.load('/content/drive/My Drive/UW/ML_596/PA5/models/resnet50_fc_4.pth'))
for param in resnet50_model.parameters():
    param.grad_requires = False

eval_analysis(resnet50_model)

Accuracy by Class
daisy          : 86.67%
dandelion      : 93.55%
rose           : 89.47%
sunflower      : 94.29%
tulip          : 97.53%
Total Accuracy = 92.30106329144246%

Confusion Matrix: 
 [[ 91   7   3   0   4]
 [  3 174   3   2   4]
 [  0   2 119   1  11]
 [  0   1   3  99   2]
 [  1   0   3   0 158]]

Total Accuracy: 
 0.9276410998552822

Classification Report: 
               precision    recall  f1-score   support

           0       0.96      0.87      0.91       105
           1       0.95      0.94      0.94       186
           2       0.91      0.89      0.90       133
           3       0.97      0.94      0.96       105
           4       0.88      0.98      0.93       162

    accuracy                           0.93       691
   macro avg       0.93      0.92      0.93       691
weighted avg       0.93      0.93      0.93       691



In [9]:
resnet101_model = torchvision.models.resnet101(pretrained=True)
# change last layer to output only 5 classes
resnet101_model.fc = nn.Linear(in_features=2048, out_features=len(classes), bias=True)

resnet101_model.load_state_dict(torch.load('/content/drive/My Drive/UW/ML_596/PA5/models/resnet101_model_fc_4_normalize.pth', map_location=torch.device('cpu')))
for param in resnet101_model.parameters():
    param.grad_requires = False

eval_analysis(resnet101_model)

Accuracy by Class
daisy          : 85.71%
dandelion      : 96.24%
rose           : 94.74%
sunflower      : 93.33%
tulip          : 94.44%
Total Accuracy = 92.89309294742232%

Confusion Matrix: 
 [[ 90   8   4   2   1]
 [  5 179   1   1   0]
 [  3   0 126   0   4]
 [  0   3   2  98   2]
 [  2   1   4   2 153]]

Total Accuracy: 
 0.934876989869754

Classification Report: 
               precision    recall  f1-score   support

           0       0.90      0.86      0.88       105
           1       0.94      0.96      0.95       186
           2       0.92      0.95      0.93       133
           3       0.95      0.93      0.94       105
           4       0.96      0.94      0.95       162

    accuracy                           0.93       691
   macro avg       0.93      0.93      0.93       691
weighted avg       0.93      0.93      0.93       691



In [ ]:
inception_v3_model = torchvision.models.inception_v3(pretrained=True)
# change last layer to output only 5 classes
inception_v3_model.fc = nn.Linear(in_features=2048, out_features=len(classes), bias=True)

inception_v3_model.load_state_dict(torch.load('/content/drive/My Drive/UW/ML_596/PA5/models/inception_v3_3.pth'))
for param in inception_v3_model.parameters():
    param.grad_requires = False

eval_analysis(inception_v3_model)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

Accuracy by Class
daisy          : 86.67%
dandelion      : 95.70%
rose           : 87.97%
sunflower      : 94.29%
tulip          : 93.21%
Total Accuracy = 91.56622140776074%

Confusion Matrix: 
 [[ 91   6   2   2   4]
 [  3 178   1   4   0]
 [  3   2 117   1  10]
 [  1   1   3  99   1]
 [  1   0   7   3 151]]

Total Accuracy: 
 0.9204052098408104

Classification Report: 
               precision    recall  f1-score   support

           0       0.92      0.87      0.89       105
           1       0.95      0.96      0.95       186
           2       0.90      0.88      0.89       133
           3       0.91      0.94      0.93       105
           4       0.91      0.93      0.92       162

    accuracy                           0.92       691
   macro avg       0.92      0.92      0.92       691
weighted avg       0.92      0.92      0.92       691



# Test Data Predictions


In [ ]:
from torchvision.io import read_image
import PIL
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os

class MyImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.image_names = os.listdir(self.img_dir)
        self.transform = transform

    def __len__(self):
      return len(self.image_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_names[idx])
        image = PIL.Image.open(img_path)
        id = self.image_names[idx]
        if self.transform:
            image = self.transform(image)
        return image, id

In [ ]:
def test_images_predictions(model_):

  model_.eval()

  y_pred_tensors = []
  image_ids_nested = []

  with torch.no_grad():
    for imgs, names in test_dataloader:    
      outputs = model_(imgs.float()) #run batch through the netork
      _, preds = torch.max(outputs, 1)
      
      y_pred_tensors.append(preds)
      image_ids_nested.append(names)


  # lists to hold numpy values
  y_pred_numpy = []

  # convert y_pred tensor to numpy list elementwise
  for p in y_pred_tensors:
    p = p.numpy()
    for x in p:
      y_pred_numpy.append(classes[x])

  image_ids = []
  for ids in image_ids_nested:
    for id in ids:
      image_ids.append(id)

  return y_pred_numpy, image_ids

In [ ]:
'''
Resnet50 Loader for test data
'''

# Instantiate Test objects of Class MyImageDataset
test_data = MyImageDataset(test_path, transform=simple_transform)

# Data Loader for test images
test_dataloader = DataLoader(test_data, batch_size=8)

resnet50_model = torchvision.models.resnet50(pretrained=True)
# change last layer to output only 5 classes
resnet50_model.fc = nn.Linear(in_features=2048, out_features=len(classes), bias=True)

# load the desired model
resnet50_model.load_state_dict(torch.load('/content/drive/My Drive/UW/ML_596/PA5/models/resnet50_fc_4.pth'))

# run the model with the test_dataloader
y_pred, img_names = test_images_predictions(resnet50_model)

In [ ]:
'''
Resnet101 Loader for test data
'''

# Instantiate Test objects of Class MyImageDataset
test_data = MyImageDataset(test_path, transform=simple_transform)

# Data Loader for test images
test_dataloader = DataLoader(test_data, batch_size=8)

resnet101_model = torchvision.models.resnet101(pretrained=True)
# change last layer to output only 5 classes
resnet101_model.fc = nn.Linear(in_features=2048, out_features=len(classes), bias=True)

# load the desired model
resnet101_model.load_state_dict(torch.load('/content/drive/My Drive/UW/ML_596/PA5/models/resnet101_model_fc_4_normalize.pth'))

# run the model with the test_dataloader
y_pred, img_names = test_images_predictions(resnet101_model)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [ ]:
'''
Inception v3 Loader for test data
'''

# Instantiate Test objects of Class MyImageDataset
test_data = MyImageDataset(test_path, transform=inception_transform)

# Data Loader for test images
test_dataloader = DataLoader(test_data, batch_size=8)

inception_v3_model = torchvision.models.inception_v3(pretrained=True)
# change last layer to output only 5 classes
inception_v3_model.fc = nn.Linear(in_features=2048, out_features=len(classes), bias=True)

inception_v3_model.load_state_dict(torch.load('/content/drive/My Drive/UW/ML_596/PA5/models/inception_v3_3.pth'))

y_pred, img_names = test_images_predictions(inception_v3_model)

# Save Predictions 

In [ ]:
import pandas as pd

out_data = {
    'Id' : (img_names),
    'Category' : (y_pred)
}

out_df = pd.DataFrame(out_data, index=None)

out_df.to_csv('/content/drive/My Drive/UW/ML_596/PA5/preds/preds4-resnet101.csv')